# N starvation 

## Which N sources allow MED4 to grow

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import subprocess
import time


In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [5]:
sns.hls_palette

<function seaborn.palettes.hls_palette(n_colors=6, h=0.01, l=0.6, s=0.65, as_cmap=False)>

In [6]:
# values from Ofaim at el.

PARAMETER_VALUES = [#"Name",     "Reaction ID",          "Lower bound", "UpperBound"
                   ["HCO3",      "HCO3EXcar",            -8,            0],
                   #["Nitrogen",  "AmmoniaEX",            -0.56,         0],
                   #["Phosphate", "FAKEOrthophosphateEX", -0.1,          0],
                   ["Light",     "LightEX",              -150,          0]]
CO2MAX = 0.82

# Block fake reactions
FAKE_TRANSPORT = ["AminosugarsOUT", "FAKEAAOUT", "FAKEABPOUT", "FAKEacpTRANS", "FAKEApoacpTRANS", "FAKEThioredoxinTRANS", 
                  "FreefattyacidsOUT", "7NMeth7carbOUT", "ArtificialproteinOUT", "FADOUT", "LipoylproteinTRANS", "MenaquinoneOUT", 
                  "NicotinateOUT", "THFpolyglutOUT", "Thiamin_dpOUT"]


In [7]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction' : rid,
        'metabolite' : m.name,
        'N' : 'N' in elements,
        'C' : 'C' in elements,
        
    }
    return pd.Series(res)

# Import model and manipulate based on Ofaim at el

In [8]:
import os
model_dpath = os.path.join('..', 'Model_files')
model_fname = 'iSO595v7.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [9]:
model = cobra.io.read_sbml_model(model_fpath)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [10]:
# manipulations copied from Ofaim at el.

remove_blocked = False

# Block H2S
model.reactions.H2SEX.lower_bound = 0

# Block fake transports
for rid in FAKE_TRANSPORT:
    model.reactions.get_by_id(rid).bounds = (0,0)

# Remove blocked reactions
if remove_blocked:
    blocked = cobra.flux_analysis.find_blocked_reactions(model, open_exchanges = True)
    print('blocked', len(blocked), blocked)
    model.remove_reactions([model.reactions.get_by_id(r_id) for r_id in blocked])

# Block maximum CO2 production
model.reactions.CO2EX.bounds = (0, CO2MAX)

for i, row in enumerate(PARAMETER_VALUES):
    # Row: Name, Reaction ID, lower bound, upper bound
    key = row[0]
    reaction_id = row[1]
    lower_bound = row[2]
    upper_bound = row[3]
    r = model.reactions.get_by_id(reaction_id)
    # Fix flux
    r.bounds = (lower_bound, upper_bound)

In [11]:
exchange_ids = [r.id for r in model.exchanges] # + ["R00024"]

In [12]:
print('reactions', len(model.reactions))
print('metabolites', len(model.metabolites))
print('genes', len(model.genes))

reactions 994
metabolites 802
genes 595


In [13]:
model

Name,COBRAModel
Memory address,0x07f8c659568b0
Number of metabolites,802
Number of reactions,994
Number of groups,55
Objective expression,1.0*BIOMASS - 1.0*BIOMASS_reverse_69053
Compartments,"Cytoplasm, Extracellular, unknownCompartment2, Periplasm, unknownCompartment5"


In [14]:
model.metabolites.get_by_id('Carbamate[c]')

Metabolite identifier,Carbamate[c]
Name,Carbamate
Memory address,0x7f8c658f6c40
Formula,CH3NO2
Compartment,c
In 4 reaction(s),"R03546, R01395, R10949, R07316"


In [15]:
cobra.Configuration()

Attribute,Description,Value
solver,Mathematical optimization solver,gurobi
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,11
cache_directory,Path for the model cache,/home/oweissberg/.cache/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


In [16]:
model.reactions.get_by_id('BIOMASS')

Reaction identifier,BIOMASS
Name,Biomass formation
Memory address,0x7f8c65726b20
Stoichiometry,104.0 ATP[c] + 0.0324116276714271 BioPool[c] + 0.0268408791654006 Bmineral[c] + 0.0559910868023904 Cell_wall[c] + 0.00111921401802897 Free_nucleic_acids[c] + 104.0 H2O[c] + 0.649245416793275... 104.0 ATP + 0.0324116276714271 BioPool + 0.0268408791654006 Trace metals and minerals + 0.0559910868023904 Cell wall Biomass precursor + 0.00111921401802897 Free nucleic acids + 104.0 H2O +...
GPR,
Lower bound,0.0
Upper bound,1000.0


# Get MED4 compound ids from KEGG and use Bigg to map to 1A3 metabolites

Apparently, MED4 metabolites are taken from KEGG with light massaging of names (copied from Shany's Refill script)

In [1]:
from bioservices.kegg import KEGG
k = KEGG()
k.organism='pmm'
paths=k.pathwayIds


Creating directory /home/oweissberg/.config/bioservices 
Creating directory /home/oweissberg/.cache/bioservices 
Welcome to Bioservices
It looks like you do not have a configuration file.
We are creating one with default values in /home/oweissberg/.config/bioservices/bioservices.cfg .
Done


In [4]:
k_compounds = k.compoundIds

In [7]:
print(k.get(k_compounds[30]))

ENTRY       C00031                      Compound
NAME        D-Glucose;
            Grape sugar;
            Dextrose;
            Glucose;
            D-Glucopyranose
FORMULA     C6H12O6
EXACT_MASS  180.0634
MOL_WEIGHT  180.1559
REMARK      Same as: D00009
COMMENT     alpha-D-Glucose [CPD:C00267]
            beta-D-Glucose [CPD:C00221]
REACTION    R00010 R00015 R00028 R00049 R00063 R00299 R00300 R00301 
            R00302 R00303 R00304 R00305 R00306 R00308 R00327 R00337 
            R00503 R00534 R00550 R00725 R00801 R00837 R00838 R00839 
            R00850 R00874 R00952 R00953 R00960 R01006 R01100 R01101 
            R01139 R01444 R01460 R01498 R01555 R01718 R01790 R01791 
            R02185 R02558 R02595 R02727 R02738 R02887 R02985 R03075 
            R03323 R03527 R03604 R03703 R03801 R03802 R03820 R04006 
            R04094 R04194 R04333 R04409 R04410 R04411 R04498 R04503 
            R04949 R04998 R05061 R05140 R05141 R05142 R05196 R05804 
            R05980 R05981 R06040 R06053 

# Compare exchange reactions/metabolites of the two models

In [75]:
exchange_ids = [r.id for r in model.exchanges] 
pro_med_df = pd.DataFrame({'mediumExchange' : exchange_ids})
pro_med_df = pro_med_df.mediumExchange.apply(get_elements)


In [76]:
#pro_df = pd.read_excel('PRO_N_sources.xlsx', sheet_name='secretion')
alt_dpath = '../../HOT1A3-model/N_Starvation/'
alt_med_df = pd.read_excel(os.path.join(alt_dpath, 'HOT1A3_medium_NCBI.xlsx'))


In [77]:
pro_med_df.head()

,reaction,metabolite,N,C
0,AminosugarsEX,Amino sugars pool,False,False
1,FAKEAAEX,amino_acid_pool_e,False,False
2,FAKEABPEX,Adenosine 3_5-bisphosphate,True,True
3,FAKEacpEX,Acyl-carrier protein,False,False
4,FAKEApoacpEX,Apo-(acyl-carrier-protein),False,False


In [78]:
alt_med_df.head()

,Unnamed: 0,reaction,metabolite,N,C,metabolite_luca,pro99,MED4 metabolite,promm
0,0,EX_14glucan_e,"1,4-alpha-D-glucan",False,True,NaN,NaN,NaN,NaN
1,1,EX_25dkglcn_e,"2,5-diketo-D-gluconate",False,True,"2,5-diketo-D-gluconate",NaN,NaN,NaN
2,2,EX_2ameph_e,2-Aminoethylphosphonate,True,True,2-Aminoethylphosphonate,NaN,NaN,NaN
3,3,EX_2m35mdntha_e,N 2 methyl 3 5 dinitrophenyl 4 methyl 3 5 di...,True,False,N 2 methyl 3 5 dinitrophenyl 4 methyl 3 5 di...,NaN,NaN,NaN
4,4,EX_2pglyc_e,2-Phosphoglycolate,False,True,NaN,NaN,NaN,NaN


In [79]:
alt_med_df.loc[(~alt_med_df.pro99.isna()) & 
               (alt_med_df.metabolite != alt_med_df['MED4 metabolite']), #&
               #(alt_med_df.metabolite_common != alt_med_df['MED4 metabolite']),
               ['metabolite','MED4 metabolite']
               
              ].values

array([['Calcium', 'Ca2+'],
       ['Chloride', 'Cl-'],
       ['CO2 CO2', 'HCO3-'],
       ['Co2+', 'Cobalt ion'],
       ['Fe2+ mitochondria', 'Fe2+'],
       ['H2O H2O', 'H2O'],
       ['Potassium', 'K+'],
       ['Magnesium', 'Magnesium cation'],
       ['Manganese', nan],
       ['Molybdate', 'Molybdenum'],
       ['Ammonium', 'Ammonia'],
       ['O2 O2', 'O2'],
       ['Phosphate', 'Orthophosphate'],
       ['Zinc', 'Zn2+']], dtype=object)

In [80]:
pro_med_df['metabolite_common'] = pro_med_df['metabolite']
alt_med_df['metabolite_common'] = alt_med_df['metabolite']

In [81]:
pro_med_df['metabolite_common'] = pro_med_df['metabolite_common'].str.replace('[e]','', regex=False)
alt_med_df['metabolite_common'] = alt_med_df['metabolite_common'].str.replace('[e]','', regex=False)
pro_med_df['metabolite_common'] = pro_med_df['metabolite_common'].str.replace('_','-', regex=False)
alt_med_df['metabolite_common'] = alt_med_df['metabolite_common'].str.replace('_','-', regex=False)
#pro_med_df['metabolite_common'] = pro_med_df['metabolite_common'].str.replace('Urea CH4N2O','Urea', regex=False)
#alt_med_df['metabolite_common'] = alt_med_df['metabolite_common'].str.replace('Urea CH4N2O','Urea', regex=False)


In [92]:
mapping_exchanges = [
    ('Ca2+','Calcium'),
    ('Fe2+ mitochondria','Fe2+'),
    ('CO','Carbon monoxide'),
    ('CO2 CO2','CO2'),
    ('Cl-','Chloride'),
    ('Co2+','Cobalt ion'),
    #['CO2 CO2', 'HCO3-'],
    ('H2O H2O', 'H2O'),
    ('K+', 'Potassium',) ,
    ('Magnesium', 'Magnesium cation'),
    ('Molybdate', 'Molybdate/Molybdenum'),
    ('Molybdenum', 'Molybdate/Molybdenum'),
    ('Ammonium', 'Ammonium/Ammonia'),
    ('Ammonia', 'Ammonium/Ammonia'),
    ('O2 O2', 'O2'),
    ('Phosphate', 'Orthophosphate'),
    ('Zinc', 'Zn2+'),
    ('S-Malate', 'S-Malate/L-Malate'),
    ('L-Malate', 'S-Malate/L-Malate'),
    ('Urea CH4N2O','Urea'),
    ('Thymidine C10H14N2O5', 'Thymidine')
]

for i, j in mapping_exchanges:
    pro_med_df.loc[pro_med_df['metabolite_common'].isin([i]), 'metabolite_common'] = j
    alt_med_df.loc[alt_med_df['metabolite_common'].isin([i]), 'metabolite_common'] = j
    

In [93]:
exchange_df = pd.merge(pro_med_df, alt_med_df, on='metabolite_common', how='outer', suffixes=['_pro', '_alt'])

In [94]:
exchange_df['map_status'] = 'mapped'
exchange_df.loc[exchange_df.metabolite_alt.isna(), 'map_status'] = 'PRO only'
exchange_df.loc[exchange_df.metabolite_pro.isna(), 'map_status'] = 'ALT only'
exchange_df.map_status.value_counts()

ALT only    202
mapped       55
PRO only     46
Name: map_status, dtype: int64

In [95]:
exchange_df.isna().sum()

reaction_pro         202
metabolite_pro       202
N_pro                202
C_pro                202
metabolite_common      0
Unnamed: 0            46
reaction_alt          46
metabolite_alt        46
N_alt                 46
C_alt                 46
metabolite_luca       72
pro99                284
MED4 metabolite      285
promm                296
map_status             0
dtype: int64

In [96]:
exchange_df[['metabolite_common','metabolite_pro', 'metabolite_alt', 'map_status', 'reaction_pro', 'reaction_alt']].to_excel('common_exchange.xlsx')

# Look for potential cross feeding

In [98]:
pro_df = pd.read_excel('data/PRO_N_sources.xlsx', sheet_name='secretion')
alt_dpath = '../../HOT1A3-model/N_Starvation/'
alt_df = pd.read_excel(os.path.join(alt_dpath, 'ALT_N_sources.xlsx'), sheet_name='secretion')
alt2_df = pd.read_excel(os.path.join(alt_dpath, 'ALT_N_sources.xlsx'), sheet_name='secretion2')


In [99]:
alt_df.head()

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,N_uptake,C_uptake,metabolite_secretion,N_secretion,C_secretion
0,0,sink_2ohph_c,0.0,66.658441,EX_cytd_e,Cytidine,True,True,2-Octaprenyl-6-hydroxyphenol,False,True
1,1,sink_4crsol_c,0.0,279.476874,EX_cytd_e,Cytidine,True,True,P-Cresol,False,True
2,3,sink_hemeO_c,0.0,65.829612,EX_cytd_e,Cytidine,True,True,Heme O C49H56FeN4O5,True,True
3,4,sink_mobd_c,0.0,1000.000000,EX_cytd_e,Cytidine,True,True,Molybdate,False,False
4,5,sink_sheme_c,0.0,95.624557,EX_cytd_e,Cytidine,True,True,Siroheme C42H36FeN4O16,True,True


In [100]:
alt2_df.loc[alt2_df.N_uptake, 'metabolite_uptake']

253      Ammonium
254      Ammonium
255      Ammonium
256      Ammonium
257      Ammonium
           ...   
20191     Cyanate
20192     Cyanate
20193     Cyanate
20194     Cyanate
20195     Cyanate
Name: metabolite_uptake, Length: 5880, dtype: object

In [101]:
alt2_df.head()

,Unnamed: 0,secretion,minimum,maximum,uptake,uptake2,metabolite_uptake,N_uptake,C_uptake,metabolite_uptake2,N_uptake2,C_uptake2,metabolite_secretion,N_secretion,C_secretion
0,0,sink_2ohph_c,0.0,68.865821,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",False,True,Ammonium,True,False,2-Octaprenyl-6-hydroxyphenol,False,True
1,1,sink_4crsol_c,0.0,308.174662,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",False,True,Ammonium,True,False,P-Cresol,False,True
2,3,sink_hemeO_c,0.0,59.261590,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",False,True,Ammonium,True,False,Heme O C49H56FeN4O5,True,True
3,4,sink_mobd_c,0.0,1000.000000,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",False,True,Ammonium,True,False,Molybdate,False,False
4,5,sink_sheme_c,0.0,77.873881,EX_14glucan_e,EX_nh4_e,"1,4-alpha-D-glucan",False,True,Ammonium,True,False,Siroheme C42H36FeN4O16,True,True


In [102]:
p_map = pro_med_df['metabolite_common']
p_map.index = pro_med_df['metabolite']
pro_df['metabolite1_uptake'] = pro_df.metabolite_uptake.replace(p_map)
pro_df['metabolite1_secretion'] = pro_df.metabolite_secretion.replace(p_map)


In [103]:
a_map = alt_med_df['metabolite_common']
a_map.index = alt_med_df['metabolite']
alt_df['metabolite1_uptake'] = alt_df.metabolite_uptake.replace(a_map)
alt_df['metabolite1_secretion'] = alt_df.metabolite_secretion.replace(a_map)
alt2_df['metabolite1_uptake'] = alt2_df.metabolite_uptake.replace(a_map)
alt2_df['metabolite1_uptake2'] = alt2_df.metabolite_uptake2.replace(a_map)
alt2_df['metabolite1_secretion'] = alt2_df.metabolite_secretion.replace(a_map)


In [104]:
alt_uptake = set(alt_df.metabolite1_uptake.unique()) | set(alt2_df.metabolite1_uptake.unique()) | set(alt2_df.metabolite1_uptake2.unique())
alt1_uptake = set(alt_df.metabolite1_uptake.unique()) 
alt2_uptake = set(alt2_df.metabolite1_uptake.unique()) | set(alt2_df.metabolite1_uptake2.unique())
pro_uptake = set(pro_df.metabolite1_uptake.unique())
alt_secretion = set(alt_df.metabolite1_secretion.unique()) | set(alt2_df.metabolite1_secretion.unique())
pro_secretion = set(pro_df.metabolite1_secretion.unique())


In [105]:
alt2_uptake = set(alt2_df.metabolite1_uptake.unique()) | set(alt2_df.metabolite1_uptake2.unique())


In [106]:
alt2_N_metabolites = (
    set(alt2_df.loc[alt2_df.N_uptake, 'metabolite1_uptake'].unique()) | 
    set(alt2_df.loc[alt2_df.N_uptake2, 'metabolite1_uptake2'].unique()) 
)
alt2_C_metabolites = (
    set(alt2_df.loc[alt2_df.C_uptake, 'metabolite1_uptake'].unique()) | 
    set(alt2_df.loc[alt2_df.C_uptake2, 'metabolite1_uptake2'].unique()) 
)



In [107]:
print(
    '\nalt2_N_metabolites', len(alt2_N_metabolites), 
    '\nalt2_C_metabolites', len(alt2_C_metabolites), 
    '\nalt2_C+N_metabolites', len(alt2_C_metabolites & alt2_N_metabolites), 
    '\nalt2_none_metabolites', len(alt2_uptake - (alt2_C_metabolites | alt2_N_metabolites)), 
    
)


alt2_N_metabolites 7 
alt2_C_metabolites 80 
alt2_C+N_metabolites 3 
alt2_none_metabolites 1


In [108]:
', '.join(sorted(alt2_N_metabolites - alt2_C_metabolites))

'2 4 6 Trinitrotoluene, Ammonium/Ammonia, Nitrate, Nitrite'

In [109]:
', '.join(sorted(alt2_C_metabolites - alt2_N_metabolites))

' R  Acetoin C4H8O2, (R)-3-Hydroxybutanoate, 1,4-alpha-D-glucan, 1-Propanol, 10 Phenyldecanoic acid, 2,5-diketo-D-gluconate, 2-Phosphoglycolate, 3-Methylbutanoic acid, 4-Hydroxyphenylacetate, 5 phenylvaleric acid, 5-Dehydro-D-gluconate, 6 Phenylhexanoic acid, 6 acetylthiohexanoic acid, 7 Phenylheptanoic acid, 8 Phenyloctanoic acid, 9 Phenylnonanoic acid, Acetaldehyde, Acetate, Acetoacetate, Beta D-Galactose, Butyrate (n-C4:0), Carbon monoxide, Cellobiose, Citrate, D-Fructose, D-Galactose, D-Galacturonate, D-Glucose, D-Glucuronate, D-Lactate, D-Mannitol, D-Mannose, D-Ribose, D-Sorbitol, Decanoate (n-C10:0), Deoxyribose C5H10O4, Dodecanoate (n-C12:0), Ethanol, Fe(III)dicitrate, Formate, Fumarate, Galactitol, Glycerol, Glycerol 2-phosphate, Glycerophosphoglycerol, Glycol, Heptanoate, Hexadecanoate (n-C16:0), Hexadecenoate (n-C16:1), Hexanoate (n-C6:0), Isobutyrate, L-Ascorbate, L-Fucose, L-Idonate, L-Lactate, Lactose C12H22O11, Linoleic acid (all cis C18:2) n-6, Maltopentaose, Maltose C12

In [110]:
', '.join(sorted(alt2_C_metabolites & alt2_N_metabolites))

'Cyanate, Phenethylamine, Urea'

In [111]:
', '.join(sorted(alt2_uptake - (alt2_C_metabolites | alt2_N_metabolites)))

'Hydrogen sulfide'

In [112]:
print(
    '\nalt->pro', len(pro_uptake & alt_secretion), 
    '\nalt_secretion', len(alt_secretion), 
    '\npro_uptake', len(pro_uptake), 
    '\npro->alt', len(alt_uptake & pro_secretion), 
    '\npro_secretion', len(pro_secretion),
    '\nalt_uptake', len(alt_uptake), 
    '\nalt1_uptake', len(alt1_uptake), 
    '\nalt2_uptake', len(alt2_uptake), 
)


alt->pro 6 
alt_secretion 53 
pro_uptake 16 
pro->alt 27 
pro_secretion 56 
alt_uptake 152 
alt1_uptake 67 
alt2_uptake 85


In [113]:
print(
    '\nalt->pro', len(pro_uptake & alt_secretion), 
    '\nalt_secretion', len(alt_secretion), 
    '\npro_uptake', len(pro_uptake), 
    '\npro->alt', len(alt_uptake & pro_secretion), 
    '\npro_secretion', len(pro_secretion),
    '\nalt_uptake', len(alt_uptake), 
    '\nalt1_uptake', len(alt1_uptake), 
    '\nalt2_uptake', len(alt2_uptake), 
)


alt->pro 6 
alt_secretion 53 
pro_uptake 16 
pro->alt 27 
pro_secretion 56 
alt_uptake 152 
alt1_uptake 67 
alt2_uptake 85


In [50]:
', '.join(sorted(alt_uptake & pro_secretion))

'Acetate, Adenine, Ammonium/Ammonia, Citrate, Ethanol, Formate, Fumarate, Glycine, Guanine, Hydrogen sulfide, L-Alanine, L-Arginine, L-Asparagine, L-Aspartate, L-Glutamate, L-Glutamine, L-Histidine, L-Isoleucine, L-Lysine, L-Proline, L-Serine, Methanol, Putrescine, Pyruvate, S_Malate/L-Malate, Succinate, Urea, Xanthosine'

In [51]:
', '.join(sorted(alt1_uptake))

'2-Aminoethylphosphonate, Adenine, Adenosine, Ala L asp L C7H11N2O5, Ala L glu L C8H13N2O5, Ala-Gln, Ala-His, Ala-L-Thr-L, Allantoin, Choline C5H14NO, Choline sulfate, Cys Gly C5H10N2O3S, Cytidine, Cytosine, D-Alanine, Gly asn L C6H11N3O4, Gly asp L C6H9N2O5, Gly glu L C7H11N2O5, Gly pro L C7H12N2O3, Gly-Cys, Gly-Gln, Gly-Phe, Gly-Tyr, Glycerophosphoserine, Glycine, Glycine betaine, Glycine-glycine-glutamine tripeptide, Glycylglycine C4H8N2O3, Glycylphenylalanine, Guanine, Hypoxanthine, Inosine, L alaninylhistidine, L alaninylthreonine, L alaninyltryptophan, L glycinylglutamate, L glycinylglutamine, L glycinylserine, L histidinylglycine, L histidinylhistidine, L-Alanine, L-Arginine, L-Asparagine, L-Aspartate, L-Carnitine, L-Cysteine, L-Glutamate, L-Glutamine, L-Histidine, L-Isoleucine, L-Lysine, L-Proline, L-Prolinylglycine, L-Serine, L-alanine-L-glutamate, L-alanylglycine, Lysine-glutamine-glycine tripeptide, NMN C11H14N2O8P, Ornithine, Putrescine, Reduced glutathione, Serine-glutamin

In [52]:
', '.join(sorted(alt2_uptake))

' R  Acetoin C4H8O2, (R)-3-Hydroxybutanoate, 1,4-alpha-D-glucan, 1-Propanol, 10 Phenyldecanoic acid, 2 4 6 Trinitrotoluene, 2,5-diketo-D-gluconate, 2-Phosphoglycolate, 3-Methylbutanoic acid, 4-Hydroxyphenylacetate, 5 phenylvaleric acid, 5-Dehydro-D-gluconate, 6 Phenylhexanoic acid, 6 acetylthiohexanoic acid, 7 Phenylheptanoic acid, 8 Phenyloctanoic acid, 9 Phenylnonanoic acid, Acetaldehyde, Acetate, Acetoacetate, Ammonium/Ammonia, Beta D-Galactose, Butyrate (n-C4:0), Carbon monoxide, Cellobiose, Citrate, Cyanate, D-Fructose, D-Galactose, D-Galacturonate, D-Glucose, D-Glucuronate, D-Lactate, D-Mannitol, D-Mannose, D-Ribose, D-Sorbitol, Decanoate (n-C10:0), Deoxyribose C5H10O4, Dodecanoate (n-C12:0), Ethanol, Fe(III)dicitrate, Formate, Fumarate, Galactitol, Glycerol, Glycerol 2-phosphate, Glycerophosphoglycerol, Glycol, Heptanoate, Hexadecanoate (n-C16:0), Hexadecenoate (n-C16:1), Hexanoate (n-C6:0), Hydrogen sulfide, Isobutyrate, L-Ascorbate, L-Fucose, L-Idonate, L-Lactate, Lactose C12H

In [53]:
', '.join(sorted(pro_secretion))

'4_Aminobenzoate, 4_Methyl_2_oxopentanoate, 5_Methylthioadenosine, Acetate, Adenine, Ammonium/Ammonia, Biotin, Citrate, Cobamide coenzyme, Dethiobiotin, Ethanol, Formate, Fumarate, Glutathione, Glycine, Glycogen, Glycolate, Guanine, Guanosine, Hydrogen peroxide, Hydrogen sulfide, L-Alanine, L-Arginine, L-Asparagine, L-Aspartate, L-Cystine, L-Glutamate, L-Glutamine, L-Histidine, L-Isoleucine, L-Leucine, L-Lysine, L-Methionine, L-Phenylalanine, L-Proline, L-Serine, L-Threonine, L-Tryptophan, L-Tyrosine, L-Valine, Lipid A disaccharide, Methanol, Pantothenate, Putrescine, Pyruvate, S-Adenosyl-4-methylthio-2-oxobutanoate, S-Methyl-5-thio-D-ribose, S_Malate/L-Malate, Spermidine, Spermine, Succinate, Thiosulfate, Thymidine, Urea, Xanthosine, sn-Glycerol 3-phosphate'

In [54]:
', '.join(sorted(pro_uptake & alt_secretion))

'Ammonium/Ammonia, Guanine, L-Aspartate, L-Serine, L-Valine, Urea'

In [55]:
', '.join(sorted(pro_uptake))

'Ammonium/Ammonia, Cyanate, Glutathione, Glycine, Guanine, Guanosine, L-Alanine, L-Arginine, L-Aspartate, L-Cystine, L-Glutamate, L-Glutamine, L-Leucine, L-Serine, L-Valine, Urea'

In [56]:
', '.join(sorted(alt_secretion))

"(2R,4S)-2-methyl-2,3,3,4-tetrahydroxytetrahydrofuran, 2-Octaprenyl-6-hydroxyphenol, 3 Hydroxy 4 acetylthiobutanoic acid, 4-Hydroxybenzoate, 5'-deoxyribose, 5-Methylthio-5-deoxy-D-ribulose 1-phosphate, Acetaldehyde, Ammonium/Ammonia, Benzoate, Cellobiose, Choline C5H14NO, D-Galactose, D-Glucose, D-Ribose, Ethanol, Ethanolamine, Fe2+, Formaldehyde, Formate, Glycerol, Glycerol 3-phosphate, Glycerophosphoglycerol, Glycine betaine, Guanine, Heme O C49H56FeN4O5, Hydrogen sulfide, Indole, Indole 3 acetate C10H8NO2, L-Aspartate, L-Homoserine, L-Lysine, L-Serine, L-Valine, Methanol, Molybdate/Molybdenum, N  2 methyl 3 5 dinitrophenyl  4 methyl 3 5 dinitroaniline, N N bis 3 5 dinitrotolyl  amine, Nitrite, Ornithine, Orthophosphate, Oxalate, P-Cresol, Phenylacetaldehyde, Protoheme C34H30FeN4O4, Selenite, Siroheme C42H36FeN4O16, Sn-Glycero-3-phosphoethanolamine, Spermidine, Sulfur, Thymine C5H6N2O2, Trans-Cinnamate, Uracil, Urea"

In [57]:
alt_uptake | pro_uptake | alt_secretion | pro_secretion

{' R  Acetoin C4H8O2',
 '(2R,4S)-2-methyl-2,3,3,4-tetrahydroxytetrahydrofuran',
 '(R)-3-Hydroxybutanoate',
 '1,4-alpha-D-glucan',
 '1-Propanol',
 '10 Phenyldecanoic acid',
 '2 4 6 Trinitrotoluene',
 '2,5-diketo-D-gluconate',
 '2-Aminoethylphosphonate',
 '2-Octaprenyl-6-hydroxyphenol',
 '2-Phosphoglycolate',
 '3 Hydroxy 4 acetylthiobutanoic acid',
 '3-Methylbutanoic acid',
 '4-Hydroxybenzoate',
 '4-Hydroxyphenylacetate',
 '4_Aminobenzoate',
 '4_Methyl_2_oxopentanoate',
 '5 phenylvaleric acid',
 "5'-deoxyribose",
 '5-Dehydro-D-gluconate',
 '5-Methylthio-5-deoxy-D-ribulose 1-phosphate',
 '5_Methylthioadenosine',
 '6 Phenylhexanoic acid',
 '6 acetylthiohexanoic acid',
 '7 Phenylheptanoic acid',
 '8 Phenyloctanoic acid',
 '9 Phenylnonanoic acid',
 'Acetaldehyde',
 'Acetate',
 'Acetoacetate',
 'Adenine',
 'Adenosine',
 'Ala L asp L C7H11N2O5',
 'Ala L glu L C8H13N2O5',
 'Ala-Gln',
 'Ala-His',
 'Ala-L-Thr-L',
 'Allantoin',
 'Ammonium/Ammonia',
 'Benzoate',
 'Beta D-Galactose',
 'Biotin',
 'Bu

In [60]:
pro_df.loc[pro_df.isna().sum(axis=1) > 0]

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,N_uptake,C_uptake,metabolite_secretion,N_secretion,C_secretion,metabolite1_uptake,metabolite1_secretion


In [61]:
alt_df.loc[alt_df.isna().sum(axis=1) > 0]

,Unnamed: 0,secretion,minimum,maximum,uptake,metabolite_uptake,N_uptake,C_uptake,metabolite_secretion,N_secretion,C_secretion,metabolite1_uptake,metabolite1_secretion
